In [1]:
from sklearn.preprocessing import StandardScaler
from sail.models.torch.tcn import TCNRegressor
import numpy as np
from sklearn.datasets import make_regression

In [2]:
# Create data as we do in tests
def regression_data(n_samples, n_features):
    X, y = make_regression(  # type: ignore
        n_samples, n_features, n_informative=10, bias=0, random_state=0
    )
    X, y = X.astype(np.float32), y.astype(np.float32).reshape(-1, 1)
    Xt = StandardScaler().fit_transform(X)
    yt = StandardScaler().fit_transform(y)
    return Xt, yt


In [3]:
# Set size of data
from tabnanny import verbose


n_samples = 1000
n_features = 12
batch_size = 20

# Get regression data
X, y = regression_data(n_samples, n_features)

# Add dummy axis, as net requires it
X = X[:, :, np.newaxis]

# Initialize network as TCNRegressor
learner_tcn = TCNRegressor(n_features, 1)

# Calculate epochs as even divisor of n_samples and batch_size for sake of easiness
epochs = n_samples // batch_size
for i in range(epochs):
    # Extract X and y values of batch_size from training set
    X_step = X[i * batch_size : (i + 1) * batch_size, :, :]
    y_step = y[i * batch_size : (i + 1) * batch_size]

    # Run partial fit
    learner_tcn.partial_fit(X_step, y_step)

# Run prediction (note: this is of course the data that we trained on,
# but only for convenience sake; we are just testing if partial_fit works)
y_prediction = learner_tcn.predict(X)


  epoch    train_loss     dur
-------  ------------  ------
      1        1.5503  0.0367
      2        0.8796  0.0111
      3        1.2281  0.0115
      4        1.2675  0.0100
      5        0.8117  0.0102
      6        0.8359  0.0097
      7        1.2760  0.0119
      8        1.4469  0.0256
      9        1.3645  0.0302
     10        1.3228  0.0263
     11        1.2328  0.0213
     12        1.1220  0.0252
     13        0.4274  0.0156
     14        1.7517  0.0160
     15        0.7725  0.0226
     16        0.9944  0.0250
     17        0.8410  0.0168
     18        1.0643  0.0157
     19        0.6399  0.0096
     20        1.4225  0.0084
     21        0.5251  0.0081
     22        0.8966  0.0095
     23        0.7269  0.0078
     24        1.6101  0.0088
     25        0.8358  0.0095
     26        0.8160  0.0091
     27        1.1293  0.0090
     28        0.7102  0.0086
     29        0.6260  0.0094
     30        1.1767  0.0093
     31        0.8314  0.0097
     32   

In [4]:
# Plot the predictions and the truth
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "vscode"

y_pred = y_prediction.flatten().tolist()
y_true = y.flatten().tolist()
x = np.arange(len(y_true))
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=x, y=y_pred, name="predicted", line=dict(color="red", width=2))
)
fig.add_trace(
    go.Scatter(x=x, y=y_true, name="actual", line=dict(color="blue", width=2))
)
fig.show()
